# Task Groups - Structured Concurrency Primitives

Task groups provide structured concurrency primitives for managing async tasks with proper lifecycle management:

**Core Components:**
- **TaskGroup**: Wrapper around anyio task groups with structured concurrency
- **create_task_group()**: Context manager for task group creation
- **start_soon()**: Start task without waiting for initialization
- **start()**: Start task and wait for initialization signal

**Key Features:**
- Structured concurrency (all tasks complete or cancel before exit)
- Automatic cleanup on cancellation or exception
- Cancel scope integration for timeout and cancellation
- Initialization protocol for service startup

In [1]:
from lionherd_core.libs.concurrency import (
    create_task_group,
    current_time,
    get_cancelled_exc_class,
    sleep,
)

## 1. Basic Task Group Usage

`create_task_group()` provides a context manager that ensures all spawned tasks complete before exiting.

In [2]:
# Simple worker task
async def worker(name: str, duration: float) -> None:
    print(f"[{name}] Starting work (duration: {duration}s)")
    await sleep(duration)
    print(f"[{name}] Completed")


# Task group ensures all tasks complete
async with create_task_group() as tg:
    tg.start_soon(worker, "Task-1", 0.1)
    tg.start_soon(worker, "Task-2", 0.05)
    tg.start_soon(worker, "Task-3", 0.15)
    print("All tasks spawned")

print("✓ All tasks completed (task group exited)")

All tasks spawned
[Task-1] Starting work (duration: 0.1s)
[Task-2] Starting work (duration: 0.05s)
[Task-3] Starting work (duration: 0.15s)
[Task-2] Completed
[Task-1] Completed
[Task-3] Completed
✓ All tasks completed (task group exited)


## 2. start_soon() - Fire and Forget

`start_soon()` spawns tasks without waiting for them to initialize. Use for background work that doesn't need coordination.

In [3]:
# Background processing
results = []


async def process_item(item: int) -> None:
    await sleep(0.01 * item)  # Simulate work
    results.append(item * 2)
    print(f"Processed item {item} -> {item * 2}")


# Clear results
results = []

async with create_task_group() as tg:
    # Spawn tasks immediately, don't wait
    for i in range(5):
        tg.start_soon(process_item, i)
    print(f"Spawned 5 tasks, results so far: {results}")
    # Tasks still running...

print(f"✓ All tasks done, results: {sorted(results)}")

Spawned 5 tasks, results so far: []
Processed item 0 -> 0
Processed item 1 -> 2
Processed item 2 -> 4
Processed item 3 -> 6
Processed item 4 -> 8
✓ All tasks done, results: [0, 2, 4, 6, 8]


### Named Tasks

Use the `name` parameter for better debugging and observability.

In [4]:
async def monitored_task(task_id: int) -> None:
    await sleep(0.05)
    print(f"Task {task_id} executing")


async with create_task_group() as tg:
    tg.start_soon(monitored_task, 1, name="worker-1")
    tg.start_soon(monitored_task, 2, name="worker-2")
    tg.start_soon(monitored_task, 3, name="worker-3")
    print("Named tasks spawned")

print("✓ All named tasks completed")

Named tasks spawned
Task 1 executing
Task 2 executing
Task 3 executing
✓ All named tasks completed


## 3. start() - Initialization Protocol

`start()` waits for a task to signal it has initialized. The task must call `task_status.started()` to signal readiness.

In [5]:
# Service that needs startup coordination
async def service_task(
    service_name: str,
    *,
    task_status=...,  # anyio injects this
) -> None:
    # Initialize service
    await sleep(0.1)  # Simulate startup
    print(f"[{service_name}] Initialized")

    # Signal ready
    task_status.started(f"{service_name} ready")

    # Continue running
    await sleep(0.2)
    print(f"[{service_name}] Shutting down")


async with create_task_group() as tg:
    # Wait for service to initialize
    status = await tg.start(service_task, "Database")
    print(f"Service started with status: {status}")

    # Can safely spawn dependent tasks now
    tg.start_soon(worker, "Dependent-Task", 0.1)

print("✓ Service lifecycle complete")

[Database] Initialized
Service started with status: Database ready
[Dependent-Task] Starting work (duration: 0.1s)
[Dependent-Task] Completed
[Database] Shutting down
✓ Service lifecycle complete


### Multiple Services with Dependencies

In [6]:
# Coordinated startup
startup_order = []


async def layered_service(
    name: str,
    startup_time: float,
    *,
    task_status=...,
) -> None:
    await sleep(startup_time)
    startup_order.append(name)
    print(f"[{name}] Ready")
    task_status.started()

    # Keep running
    await sleep(0.5)


# Clear
startup_order = []

async with create_task_group() as tg:
    # Start services in dependency order
    await tg.start(layered_service, "Database", 0.05)
    print("Database ready, starting cache...")

    await tg.start(layered_service, "Cache", 0.03)
    print("Cache ready, starting API...")

    await tg.start(layered_service, "API", 0.02)
    print("API ready, all services running")

print(f"✓ Startup order: {startup_order}")

[Database] Ready
Database ready, starting cache...
[Cache] Ready
Cache ready, starting API...
[API] Ready
API ready, all services running
✓ Startup order: ['Database', 'Cache', 'API']


## 4. Cancel Scope - Timeout and Cancellation

Task groups have a cancel scope for implementing timeouts and coordinated cancellation.

In [7]:
# Timeout for task group
async def slow_task(task_id: int) -> None:
    try:
        print(f"Task {task_id} starting")
        await sleep(5.0)  # Too slow
        print(f"Task {task_id} completed")
    except get_cancelled_exc_class():
        print(f"Task {task_id} cancelled")
        raise


try:
    async with create_task_group() as tg:
        # Set timeout on the task group
        tg.cancel_scope.deadline = current_time() + 0.2

        tg.start_soon(slow_task, 1)
        tg.start_soon(slow_task, 2)
        tg.start_soon(slow_task, 3)
except TimeoutError:
    print("✓ Task group cancelled due to timeout")

Task 1 starting
Task 2 starting
Task 3 starting
Task 3 cancelled
Task 2 cancelled
Task 1 cancelled


### Manual Cancellation

In [8]:
# Conditional cancellation
found_result = None


async def search_task(task_id: int) -> None:
    global found_result
    try:
        for i in range(10):
            await sleep(0.02)
            if task_id == 2 and i == 3:
                found_result = f"Task {task_id} found target!"
                print(found_result)
                return
    except get_cancelled_exc_class():
        print(f"Task {task_id} cancelled")
        raise


# Clear
found_result = None

async with create_task_group() as tg:
    tg.start_soon(search_task, 1)
    tg.start_soon(search_task, 2)
    tg.start_soon(search_task, 3)

    # Monitor for result
    while not found_result:
        await sleep(0.01)

    # Cancel all tasks once we have result
    print("Cancelling remaining tasks...")
    tg.cancel_scope.cancel()

print(f"✓ Result: {found_result}")

Task 2 found target!
Cancelling remaining tasks...
Task 1 cancelled
Task 3 cancelled
✓ Result: Task 2 found target!


## 5. Exception Handling

If any task raises an exception, all tasks in the group are cancelled and the exception propagates.

In [9]:
# Task that fails
async def failing_task(task_id: int, should_fail: bool) -> None:
    try:
        await sleep(0.05)
        if should_fail:
            raise ValueError(f"Task {task_id} failed")
        print(f"Task {task_id} completed")
    except get_cancelled_exc_class():
        print(f"Task {task_id} cancelled due to sibling failure")
        raise


try:
    async with create_task_group() as tg:
        tg.start_soon(failing_task, 1, False)
        tg.start_soon(failing_task, 2, True)  # This will fail
        tg.start_soon(failing_task, 3, False)
except* ValueError as eg:
    print(f"\n✓ Task group aborted, caught {len(eg.exceptions)} exception(s):")
    for exc in eg.exceptions:
        print(f"  - {exc}")

Task 1 completed
Task 3 completed

✓ Task group aborted, caught 1 exception(s):
  - Task 2 failed


### Multiple Exceptions

In [10]:
# Multiple tasks fail simultaneously
async def multi_fail_task(task_id: int, error_type: type[Exception] | None) -> None:
    await sleep(0.02)
    if error_type:
        raise error_type(f"Task {task_id} error")


try:
    async with create_task_group() as tg:
        tg.start_soon(multi_fail_task, 1, ValueError)
        tg.start_soon(multi_fail_task, 2, TypeError)
        tg.start_soon(multi_fail_task, 3, None)  # Succeeds but cancelled
except* (ValueError, TypeError) as eg:
    print(f"✓ Caught multiple exception types ({len(eg.exceptions)} total):")
    for exc in eg.exceptions:
        print(f"  - {type(exc).__name__}: {exc}")

✓ Caught multiple exception types (2 total):
  - ValueError: Task 1 error
  - TypeError: Task 2 error


## 6. Practical Patterns

Common patterns using task groups in real applications.

### Pattern: Worker Pool

In [11]:
# Fixed-size worker pool with simple queue pattern
async def worker_pool_example() -> None:
    processed_items = []

    async def worker(worker_id: int, items: list[str]) -> None:
        for item in items:
            await sleep(0.02)  # Process item
            processed_items.append(f"Worker {worker_id} processed: {item}")
            print(f"Worker {worker_id} processed: {item}")

    # Distribute work among workers
    items = [f"item-{i}" for i in range(12)]
    chunk_size = 4

    async with create_task_group() as tg:
        # Spawn workers with their assigned items
        for i in range(3):
            start = i * chunk_size
            end = start + chunk_size
            tg.start_soon(worker, i, items[start:end])

    print(f"✓ All work processed ({len(processed_items)} items)")


await worker_pool_example()

Worker 0 processed: item-0
Worker 1 processed: item-4
Worker 2 processed: item-8
Worker 0 processed: item-1
Worker 1 processed: item-5
Worker 2 processed: item-9
Worker 0 processed: item-2
Worker 1 processed: item-6
Worker 2 processed: item-10
Worker 0 processed: item-3
Worker 1 processed: item-7
Worker 2 processed: item-11
✓ All work processed (12 items)


### Pattern: Service Manager

In [12]:
# Manage multiple long-running services
class ServiceManager:
    def __init__(self) -> None:
        self.services_ready: list[str] = []

    async def run_service(
        self,
        name: str,
        *,
        task_status=...,
    ) -> None:
        # Startup
        await sleep(0.05)
        self.services_ready.append(name)
        print(f"[{name}] Started")
        task_status.started()

        # Run until cancelled
        try:
            while True:
                await sleep(0.1)
        except get_cancelled_exc_class():
            print(f"[{name}] Shutting down gracefully")
            raise

    async def run(self) -> None:
        async with create_task_group() as tg:
            # Start all services
            await tg.start(self.run_service, "Database")
            await tg.start(self.run_service, "Cache")
            await tg.start(self.run_service, "API")

            print(f"All services ready: {self.services_ready}")

            # Run for a bit
            await sleep(0.2)

            # Graceful shutdown
            print("Initiating shutdown...")
            tg.cancel_scope.cancel()


manager = ServiceManager()
await manager.run()
print("✓ All services stopped")

[Database] Started
[Cache] Started
[API] Started
All services ready: ['Database', 'Cache', 'API']
Initiating shutdown...
[Database] Shutting down gracefully
[API] Shutting down gracefully
[Cache] Shutting down gracefully
✓ All services stopped


### Pattern: Nested Task Groups

In [13]:
# Hierarchical task organization
async def subtask(group: str, task_id: int) -> None:
    await sleep(0.02)
    print(f"  [{group}] Subtask {task_id} done")


async def task_with_subtasks(task_id: int) -> None:
    print(f"Task {task_id} spawning subtasks...")
    async with create_task_group() as subtg:
        subtg.start_soon(subtask, f"Task-{task_id}", 1)
        subtg.start_soon(subtask, f"Task-{task_id}", 2)
        subtg.start_soon(subtask, f"Task-{task_id}", 3)
    print(f"Task {task_id} completed all subtasks")


async with create_task_group() as tg:
    tg.start_soon(task_with_subtasks, 1)
    tg.start_soon(task_with_subtasks, 2)

print("✓ All tasks and subtasks completed")

Task 1 spawning subtasks...
Task 2 spawning subtasks...
  [Task-1] Subtask 1 done
  [Task-1] Subtask 2 done
  [Task-1] Subtask 3 done
  [Task-2] Subtask 1 done
  [Task-2] Subtask 2 done
  [Task-2] Subtask 3 done
Task 1 completed all subtasks
Task 2 completed all subtasks
✓ All tasks and subtasks completed


## 7. Comparison with asyncio.TaskGroup

lionherd's TaskGroup wraps anyio for cross-platform async compatibility.

In [14]:
# Key differences:

# asyncio.TaskGroup (Python 3.11+):
# - create_task() method
# - asyncio-specific
# - No start() protocol

# lionherd TaskGroup:
# - start_soon() method (anyio naming)
# - Cross-platform (asyncio, trio)
# - start() protocol for initialization
# - Direct cancel_scope access


async def demo_task() -> None:
    await sleep(0.05)
    print("Task completed")


# lionherd pattern (cross-platform)
async with create_task_group() as tg:
    tg.start_soon(demo_task)
    # Works on asyncio, trio, etc.

print("✓ Cross-platform structured concurrency")

Task completed
✓ Cross-platform structured concurrency


## Summary Checklist

**Task Group Essentials:**
- ✅ `create_task_group()` for structured concurrency context
- ✅ `start_soon()` spawns tasks without waiting (fire-and-forget)
- ✅ `start()` waits for task initialization via `task_status.started()`
- ✅ `cancel_scope` property for timeout and cancellation control
- ✅ Named tasks via `name` parameter for debugging

**Lifecycle Guarantees:**
- ✅ All tasks complete or cancel before context exit
- ✅ Exception in any task cancels all siblings
- ✅ Automatic cleanup on cancellation
- ✅ ExceptionGroup for multiple task failures

**Practical Patterns:**
- ✅ Worker pools with shared queues
- ✅ Service managers with coordinated startup/shutdown
- ✅ Nested task groups for hierarchical organization
- ✅ Timeout and deadline management
- ✅ Conditional cancellation based on results

**Next Steps:**
- See `_patterns` for high-level concurrency patterns (gather, race, bounded_map)
- See `_primitives` for CapacityLimiter and other synchronization primitives
- See `_cancel` for deadline and cancellation utilities